In [17]:
# Cell 0

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [18]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [19]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [20]:
# Cell 3

wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("keystores/balanced_test.json", key_data)
print(icon_service.get_balance(wallet.get_address())/10**18)
print(icon_service.get_balance(btest_wallet.get_address())/10**18)

776338449.6328553
2955900.0951286


In [21]:
print(wallet.get_address())
print(icon_service.get_balance(wallet.get_address()) / 10**18)

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb
776338449.6328553


In [22]:
print(btest_wallet.get_address())
print(icon_service.get_balance(btest_wallet.get_address()) / 10**18)

hx3f01840a599da07b0f620eeae7aa9c574169a4be
2955900.0951286


In [23]:
user1 = KeyWallet.load("keystores/user1.json","HelloWorld@1234")
# btest_wallet = KeyWallet.load("./balanced_test.json","HelloWorld@1234")

print(icon_service.get_balance(user1.get_address())/10**18)
print(user1.get_address())

# test2 = hx7a1824129a8fe803e45a3aae1c0e060399546187
private = "0a354424b20a7e3c55c43808d607bddfac85d033e63d7d093cb9f0a26c4ee022"
user2 = KeyWallet.load(bytes.fromhex(private))
print(icon_service.get_balance(user2.get_address())/10**18)
print(user2.get_address())

59751.6306396
hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db
3199.2083936
hx7a1824129a8fe803e45a3aae1c0e060399546187


In [24]:
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxbbd36ca8d91aefbe1060c3be62fed4d1b848ca85'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxd8e05c1280bc2c32bf53ff61f3bb2e2ecc7d6df5'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx7c617e3fca4ba06b6ad203ce113245ae96a9d91e'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx70cd5c86f0182d5ac0bd224562d929cd968d9132'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxeb91bc377b0620356787d9c9eb68152eb0c62d8a'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxfd7511ece084744154fed19dc34732681ad078e6'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx9a3161c778eee2d5758371d3548c5599f76704ec'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx238cd1a1e3a9702d6c9c6dc130719472164db376'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx7171e2f5653c1b9c000e24228276b8d24e84f10d'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxcff8bf80ab213fa9bbb350636a4d68f5cb4fd9c1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx4c1beaa71b9377100c810c46059ddf5f3da37602'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx3825a86d52c5baf188ff29aa6a7fc2467e285885'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx140b49ea041457ebc4cd5e199f5723916bb50021'}}

In [25]:
# Cell 6
# Define deploy and send_tx functions

def compress():
    """
    Compress all SCORE folders in the core_contracts and toekn_contracts folders
    """
    deploy = list(contracts.keys())[:]
    for directory in {"core_contracts", "token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 4000100000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res.get('scoreAddress', '')

def send_tx(dest, value, method, params, wallet):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Calling {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    return res

def deploy_all(wallet):
    """
    Compress, Deploy and Configure all SCOREs
    """
    compress()

    deploy = list(contracts.keys())[:]
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')

    governance = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    contracts['governance']['SCORE'] = governance
    for score in deploy:
        contracts[score]['SCORE'] = deploy_SCORE(contracts[score], {'_governance': governance}, wallet, 0)
    contracts['staking']['SCORE'] = deploy_SCORE(contracts['staking'], {}, wallet, 0)
    contracts['sicx']['SCORE'] = deploy_SCORE(contracts['sicx'], {'_admin': contracts['staking']['SCORE']}, wallet, 0)

    config = list(contracts.keys())[:]
    config.remove('governance')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}

    txns = [{'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}},
            {'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}},
            {'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}}]

    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Reading {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    print(icon_service.call(call))
    return icon_service.call(call)


In [26]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

results = {}
deploy_all(btest_wallet)
print('------------------------------------------------------------------------------------------------------------------')
print(contracts)
print(get_scores_json(contracts))

core_contracts/governance.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

core_contracts/loans.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

core_contracts/dividends.zip
Status: 1

core_contracts/reserve.zip


...

Retrying in 1 seconds...
Status: 1

------------------------------------------------------------------------------------------------------------------
Calling setSicxAddress, with parameters {'_address': 'cxe033a391ad0e124db500ec2d734abe29a9e5e3ec'} on the staking contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling setAddresses, with parameters {'_addresses': {'loans': 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab', 'staking': 'cx7fb5a0548ed8048983fccf4803ff9b60a422382f', 'dividends': 'cxf60d5b2ae6aee8ea6efffca4ab0ac79d450741d5', 'reserve': 'cx07fbf487cae6a41028f30cd888f587c10818caa0', 'daofund': 'cxc8137987ec6ade212052de2dd04f5c1cc38c8de1', 'rewards': 'cx3e940dd067d7fe21557552972f45de5ecc0cc0fc', 'dex': 'cx5b3b650487333e8bfa00416f02d41c95e9a7f508', 'oracle': 'cx7171e2f5653c1b9c000e24228276b8d24e84f10d', 'sicx': 'cxe033a391ad0e124db500ec2d734abe29a9e5e3ec', 'bnUSD': 'cx03d4c178da96e7eda17c2684cb4623e20daf9317', 'baln': 'cx95638180a76045184e6f5b37b50c6aae372767f3', 'bwt': 'cx91d2b7a0ab566c3fc458c1efa54598e52ec04d8a'}} on the governance contract

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab', 'indexed': ['AssetAdded(Address,str,bool)', 'cxe033a391ad0e124db500ec2d734abe29a9e5e3ec', 'sICX', '0x1'], 'data': []} 

{'scoreAddress': 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab', 'indexed': ['AssetAdded(Address,str,bool)', 'cx03d4c178da96e7eda17c2684cb4623e20daf9317', 'bnUSD', '0x0'], 'data': []} 

{'scoreAddress': 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab', 'indexed': ['AssetAdded(Address,str,bool)', 'cx95638180a76045184e6f5b37b50c6aae372767f3', 'BALN', '0x1'], 'data': []} 

{'scoreAddress': 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab', 'indexed': ['ContractActive(str,str)', 'Loans'], 'data': ['Active']} 

------------------------------------------------------------------------------------------------------------------
{'loans': {'zip': 'core_contracts/loans.zip', 'SCORE': 'cxbe9c1c

# Cell 8
# Deploy or Update a single SCORE

compress()
update = 0
contract = contracts['reserve']
params = {}
params = {'_governance': contracts['governance']['SCORE']}
deploy_SCORE(contract, params, btest_wallet, update)

In [27]:
# 1.Add 500icx collateral and mint 50 bnusd of loans from account user2.
# 2.call add_test_position method and add bnusd until account is in dead market state from account user1.
# 3.after this the asset bnusd will be dead

send_tx('loans', 500 * ICX , 'addCollateral', {'_asset': 'bnUSD', '_amount': 50 * ICX}, user2)
call_tx('loans', 'getAccountPositions', {'_owner': user2.get_address()})
send_tx('loans', 100 * ICX , 'addCollateral', {'_asset': '', '_amount': 0}, user1)
send_tx('loans', 0, 'add_bad_test_position', {}, user1)
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})

------------------------------------------------------------------------------------------------------------------
Calling addCollateral, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab', 'cx7fb5a0548ed8048983fccf4803ff9b60a422382f', '0x1b1ae4d6e2ef500000'], 'data': []} 

{'scoreAddress': 'cxe033a391ad0e124db500ec2d734abe29a9e5e3ec', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxe033a391ad0e124db500ec2d734abe29a9e5e3ec', 'hx7a1824129a8fe803e45a3aae1c0e060399546187', '0x1b1ae4d6e2ef500000'], 'data': ['0x7b226d6574686f64223a225f6465706f7369745f616e645f626f72726f77222c22706172616d73223a7b225f73656e646572223a22687837613138323431323961386665383033653435613361616531633065303630333939353436313837222c225f6173736574223a22626e555344222c225f616d6f756e74223a35303030303030303030303030303030303030307d7d']} 

{'scoreAddress': 'cxe033a391ad0e124db500ec2d734abe29a9e5e3ec', 'indexed': ['Mint

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab', 'cx7fb5a0548ed8048983fccf4803ff9b60a422382f', '0x56bc75e2d63100000'], 'data': []} 

{'scoreAddress': 'cxe033a391ad0e124db500ec2d734abe29a9e5e3ec', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxe033a391ad0e124db500ec2d734abe29a9e5e3ec', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', '0x56bc75e2d63100000'], 'data': ['0x7b226d6574686f64223a225f6465706f7369745f616e645f626f72726f77222c22706172616d73223a7b225f73656e646572223a22687865363264313166613139613065383537356164393266303662633866643432656462666532376462222c225f6173736574223a22222c225f616d6f756e74223a307d7d']} 

{'scoreAddress': 'cxe033a391ad0e124db500ec2d734abe29a9e5e3ec', 'indexed': ['Mint(Address,int,bytes)', 'hxe62d11fa19a0e8575ad92f06b

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx03d4c178da96e7eda17c2684cb4623e20daf9317', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx03d4c178da96e7eda17c2684cb4623e20daf9317', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', '0xad78ebc5ac6200000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx03d4c178da96e7eda17c2684cb4623e20daf9317', 'indexed': ['Mint(Address,int,bytes)', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'], 'data': ['0xad78ebc5ac6200000', '0x4e6f6e65']} 

------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x2', 'created': '0x5be4433aac79b', 'addre

{'pos_id': '0x2',
 'created': '0x5be4433aac79b',
 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db',
 'snap_id': '0x1',
 'snaps_length': '0x1',
 'last_snap': '0x1',
 'first day': '0x1',
 'assets': {'sICX': '0x56bc75e2d63100000', 'bnUSD': '0xad78ebc5ac6200000'},
 'replay_index': '0x0',
 'events_behind': '0x0',
 'total_debt': '0x67ba918e7881db218',
 'ratio': '0xb9ab7616a3414dd',
 'standing': 'Liquidate'}

In [28]:
# call_tx('loans', 'checkDebts', {})
# send_tx('loans', 0, 'add_bad_test_position', {}, user1)
call_tx('loans', 'checkDebts', {})
send_tx('loans', 0, 'liquidate', {'_owner': user1.get_address()}, user1)
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
call_tx('loans', 'checkDebts', {})


------------------------------------------------------------------------------------------------------------------
Reading checkDebts, with parameters {} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'system': {'debts': '0xd9462c6cb4b5a0000', 'bad_debt': '0x0', 'supply': '0xd9462c6cb4b5a0000', 'diff': '0x0', 'remaining': '0x0'}, 'events': {}}
------------------------------------------------------------------------------------------------------------------
Calling liquidate, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx03d4c178da96e7eda17c2684cb4623e20daf9317', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cx7171e2f5653c1b9c000e24228276b8d24e84f10d'], 'data': ['0x84c5d90848544bb']} 

{'scoreAddress': 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab', 'indexed': ['PositionStanding(Address,str,str,str)', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'Liquidate'], 'data': ['0.8361823098521571', 'Position up to date.']} 

{'scoreAddress': 'cxe033a391ad0e124db500ec2d734abe29a9e5e3ec', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', '0x94c51733f830000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab', 'indexed': ['TokenTransfer(Address,int,str)', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', '0x94c

{'system': {'debts': '0x2bcd40a70853a0000',
  'bad_debt': '0xad78ebc5ac6200000',
  'supply': '0xd9462c6cb4b5a0000',
  'diff': '0x0',
  'remaining': '0x0'},
 'events': {}}

In [36]:
# send_tx('loans', 500 * ICX , 'addCollateral', {'_asset': 'bnUSD', '_amount': 50 * ICX}, btest_wallet)
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x2', 'created': '0x5be2c15693ff5', 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'snap_id': '0x1', 'snaps_length': '0x1', 'last_snap': '0x1', 'first day': '0x1', 'assets': {'sICX': '0x0', 'bnUSD': '0x0'}, 'replay_index': '0x0', 'events_behind': '0x0', 'total_debt': '0x0', 'ratio': '0x0', 'standing': 'Zero'}


{'pos_id': '0x2',
 'created': '0x5be2c15693ff5',
 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db',
 'snap_id': '0x1',
 'snaps_length': '0x1',
 'last_snap': '0x1',
 'first day': '0x1',
 'assets': {'sICX': '0x0', 'bnUSD': '0x0'},
 'replay_index': '0x0',
 'events_behind': '0x0',
 'total_debt': '0x0',
 'ratio': '0x0',
 'standing': 'Zero'}

In [30]:
#  Now we try to revive the dead market
# 1. Test with retiring less than enough to revive the market with bad_debt 550.25 bnusd 
# i.e user2 retires 100 bnusd

test_cases =  { 
    
    "stories": [ {
      "description": "btest_wallet retire 50 bnusd",
      "actions": {
          "sender":"user2",
          "first_meth": "transfer",
          "second_meth":"transfer",
          "deposited_icx":"0",
          "first_params":{'_to': btest_wallet.get_address(), '_value': 50 * ICX},
          "second_params":{"method": "_retire_asset", "params": {},'_to': contracts['loans']['SCORE'], '_value': 50 * ICX},
          "expected_bnusd_wallet":"20700000000000000000"
          
      }
    },
        {
            "description": "btest_wallet retire 50 bnusd",
              "actions": {
                  "sender":"user2",
                  "first_meth": "transfer",
                  "second_meth":"transfer",
                  "deposited_icx":"0",
                  "first_params":{'_to': btest_wallet.get_address(), '_value': 50 *ICX},
                  "second_params":{"method": "_retire_asset", "params": {},'_to': contracts['loans']['SCORE'], '_value': 50 * ICX},
                  "expected_bnusd_wallet":"20700000000000000000"

      }
        }

  ]
}

for case in test_cases['stories']:
    print(case['description'])
    _to = contracts['bnUSD']['SCORE']
    meth1 = case['actions']['first_meth']
    meth2 = case['actions']['second_meth']
    val = int(case['actions']['deposited_icx'])
    data1 = case['actions']['first_params']
    first_params = {"_to": data1['_to'], "_value": data1['_value']}
    
    data2 = case['actions']['second_params']
    params = {"method": data2['method'], "params": data2['params']}
    data = json.dumps(params).encode("utf-8")
    second_params = {'_to': data2['_to'], '_value': data2['_value'], '_data': data}


    transaction = CallTransactionBuilder()\
        .from_(user1.get_address())\
        .to(_to)\
        .value(val)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(meth1)\
        .params(first_params)\
        .build()
    signed_transaction = SignedTransaction(transaction, user1)
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

    transaction = CallTransactionBuilder()\
        .from_(btest_wallet.get_address())\
        .to(_to)\
        .value(val)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(meth2)\
        .params(second_params)\
        .build()
    signed_transaction = SignedTransaction(transaction, btest_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(1)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    
    tes= call_tx('loans', 'checkDebts', {})
    if tes['system']['bad_debt'] != hex(0):
        print("Market still dead")
    else:
        print("Market revived")
    

btest_wallet retire 50 bnusd
Status: 1
{'scoreAddress': 'cx03d4c178da96e7eda17c2684cb4623e20daf9317', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cx7171e2f5653c1b9c000e24228276b8d24e84f10d'], 'data': ['0x84c5d90848544bb']} 

{'scoreAddress': 'cx03d4c178da96e7eda17c2684cb4623e20daf9317', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab', 'cx03d4c178da96e7eda17c2684cb4623e20daf9317', '0x2b26b8169c7af0000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx03d4c178da96e7eda17c2684cb4623e20daf9317', 'indexed': ['Burn(Address,int)', 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab'], 'data': ['0x2b26b8169c7af0000']} 

{'scoreAddress': 'cxe033a391ad0e124db500ec2d734abe29a9e5e3ec', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e0337ab', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0x1c61fe6b7edc9eadf'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxbe9c1c46b8a7bdfb41c52c4e60d04e367e033

In [122]:
# call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})

send_tx('staking',  800*ICX, 'stakeICX', {'_to':contracts['reserve']['SCORE']}, user1)

# send_tx('loans', 500 * ICX , 'addCollateral', {'_asset': 'bnUSD', '_amount': 50 * ICX}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling stakeICX, with parameters {'_to': 'cxe74fb8364f4edbd1e88af6cdfeeb7fe939b743d2'} on the staking contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx098ce1ed177b2e3f5e3416b4cb61d2a85c084e90', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx098ce1ed177b2e3f5e3416b4cb61d2a85c084e90', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', '0x2b5e3af16b18800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx098ce1ed177b2e3f5e3416b4cb61d2a85c084e90', 'indexed': ['Mint(Address,int,bytes)', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'], 'data': ['0x2b5e3af16b18800000', '0x4e6f6e65']} 

{'scoreAddress': 'cx098ce1ed177b2e3f5e3416b4cb61d2a85c084e90', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'cxe74fb8364f4edbd1e88af6cdfeeb7fe939b743d2', '0x2b5e3af16b18800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx4c19ea164684d8b9e52d8cb89a20a28998a31edc', 'indexed': ['TokenTransfer(Address,int,str)', 'cxe74fb8364f4edbd1e88af6cdfeeb7fe939b743

{'txHash': '0xc1f65cf10d58f92a21d59409e8756591db80974e5d0df0e012becce38cfe1e48',
 'blockHeight': 1533442,
 'blockHash': '0xa083f0deade50430501c89eefad02b8650c786adff84fbe6eac699d7ef577a38',
 'txIndex': 1,
 'to': 'cx4c19ea164684d8b9e52d8cb89a20a28998a31edc',
 'stepUsed': 532120,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 532120,
 'eventLogs': [{'scoreAddress': 'cx098ce1ed177b2e3f5e3416b4cb61d2a85c084e90',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx098ce1ed177b2e3f5e3416b4cb61d2a85c084e90',
    'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db',
    '0x2b5e3af16b18800000'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cx098ce1ed177b2e3f5e3416b4cb61d2a85c084e90',
   'indexed': ['Mint(Address,int,bytes)',
    'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'],
   'data': ['0x2b5e3af16b18800000', '0x4e6f6e65']},
  {'scoreAddress': 'cx098ce1ed177b2e3f5e3416b4cb61d2a85c084e90',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27

Raised JSONRPCException while returning the custom response. Error message: {'code': -32006, 'message': 'Type mismatch: value=hx7a1824129a8fe803e45a3aae1c0e060399546187 type_hint=typing.List[str]'}


JSONRPCException: {'code': -32006, 'message': 'Type mismatch: value=hx7a1824129a8fe803e45a3aae1c0e060399546187 type_hint=typing.List[str]'} (Json rpc error)

In [16]:
# send_tx('rewards',  0, 'claimRewards', {}, user1)

call_tx('baln', 'balanceOf', {'_owner': user2.get_address()})


------------------------------------------------------------------------------------------------------------------
Reading balanceOf, with parameters {'_owner': 'hx7a1824129a8fe803e45a3aae1c0e060399546187'} on the baln contract.
------------------------------------------------------------------------------------------------------------------
0x0


'0x0'